In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset
train_dataset = dsets.MNIST(root='/home/damvantai/Documents/data/', train=True, transform=transforms.ToTensor(),download=True)

In [24]:
train_dataset

In [4]:
test_dataset = dsets.MNIST(root='/home/damvantai/Documents/data/',
                          train=False,
                          transform=transforms.ToTensor())

In [5]:
# Data Loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,
                                          shuffle=True)

In [6]:
train_loader

In [7]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,
                                         shuffle=False)

In [23]:
train_loader

In [8]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7 * 7 * 32, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [9]:
cnn = CNN()

In [11]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch: [1/5], Step: [100/600], Loss: 0.1129
Epoch: [1/5], Step: [200/600], Loss: 0.0757
Epoch: [1/5], Step: [300/600], Loss: 0.1080
Epoch: [1/5], Step: [400/600], Loss: 0.0613
Epoch: [1/5], Step: [500/600], Loss: 0.0661
Epoch: [1/5], Step: [600/600], Loss: 0.0283
Epoch: [2/5], Step: [100/600], Loss: 0.0168
Epoch: [2/5], Step: [200/600], Loss: 0.0209
Epoch: [2/5], Step: [300/600], Loss: 0.0422
Epoch: [2/5], Step: [400/600], Loss: 0.0362
Epoch: [2/5], Step: [500/600], Loss: 0.0838
Epoch: [2/5], Step: [600/600], Loss: 0.0347
Epoch: [3/5], Step: [100/600], Loss: 0.0273
Epoch: [3/5], Step: [200/600], Loss: 0.0079
Epoch: [3/5], Step: [300/600], Loss: 0.1195
Epoch: [3/5], Step: [400/600], Loss: 0.0365
Epoch: [3/5], Step: [500/600], Loss: 0.0226
Epoch: [3/5], Step: [600/600], Loss: 0.0942
Epoch: [4/5], Step: [100/600], Loss: 0.0305
Epoch: [4/5], Step: [200/600], Loss: 0.0576
Epoch: [4/5], Step: [300/600], Loss: 0.0079
Epoch: [4/5], Step: [400/600], Loss: 0.0422
Epoch: [4/5], Step: [500/600], L

In [28]:
# Test the Model
cnn.eval()    # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
#     print(images)
    correct += (predicted == labels).sum()
print(correct)
print(total)

9865
10000


In [13]:
print("Test Accuracy of the model on the 10000 test images: %d %%" % (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98 %


In [16]:
print(test_loader)

In [20]:
type(test_loader)

torch.utils.data.dataloader.DataLoader

In [29]:
test_loader